- learning_rate = 3e-4
- adam_epsilon = 1e-8
- warmup_ratio = 0.05 
- weight_decay = 0.0005

LR=3e-4_ae=1e-8_wr=0.05_wd=0.0005

In [15]:
import torch
import pandas as pd 
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from keras.preprocessing.sequence import pad_sequences
import IPython

torch.cuda.is_available() # Check that PyTorch sees it

True

In [23]:
data_path = "/home/lieberze/DP/Thesis/05_model_training/data/512_bp_for_encoding/NEW/All_equal_shuffled_all_the_same_length_512.txt"       

In [68]:
! cat $data_path | wc -l

6674853


In [17]:
# https://docs.wandb.ai/guides/track/jupyter

# !pip install wandb -qqq
import wandb
wandb.login()

True

In [18]:
wandb.init(project="small_test",
           config={
               # "batch_size": 128,
               # "learning_rate": 0.01,
               # "dataset": "CIFAR-100",
           })

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [19]:
# https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#module-tokenizers.processors
from tokenizers import ByteLevelBPETokenizer
from tokenizers import Tokenizer

path = "/home/lieberze/DP/Thesis/02_tokenizery_new_data/02_ByteLevelBPE/All_genomes_sample/All_512/5000/"
tokenizer = ByteLevelBPETokenizer(
    path + "vocab.json",
    path + "merges.txt",
)

# https://huggingface.co/docs/transformers/preprocessing
from tokenizers.processors import BertProcessing
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>"))
)
# tokenizer.enable_padding(direction='right', pad_id=1, pad_type_id=1, pad_token='<pad>', length=128, pad_to_multiple_of=None)
# tokenizer.enable_truncation(max_length=128)
# tokenizer.add_special_tokens(['<mask>'])
tokenizer.save("byte-level-bpe.tokenizer.json", pretty=True)

from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained(path)
tokenizer.vocab_size

file /home/lieberze/DP/Thesis/02_tokenizery_new_data/02_ByteLevelBPE/All_genomes_sample/All_512/5000/config.json not found


5000

In [20]:
# tokenizer.tokenize("actttttgaghc"), tokenizer.encode("actttttgaghc")
# tokenizer.encode("acccctag", max_length=128, padding="max_length", truncation=True)
# tokenizer.batch_encode_plus(["actttttgaghc", "ahi"])
# len(tokenizer.encode("actttttgaghc").ids)

https://aigeekprogrammer.com/data-preparation-with-dataset-and-dataloader-in-pytorch/
PyTorch offers two classes for data processing: torch.utils.data.Dataset and torch.utils.data.DataLoader. To simplify somewhat, **Dataset**‘s task is to retrieve a single data point together with its label from a dataset, while **DataLoader** wraps the data retrieved by Dataset with an iterator, ensures that it is served in batches, runs in multiple threads to speed up the retrieval of data for training if necessary, and supports such operations as data shuffling.

dataloader
pytorch - datasety 
nezapomenout train/test => eval_dataset

run_language_modeling.py 
ne pres radky ale pres batche

https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

    # data_set(data_values, labels) tohle pak dam modelu (jako objekt)
    # pak train/test => posílat modelu
    # uzpusobit velikost batche tak, jak nejvíc jde
    # pak spustit a zkusit, zda se trénuje (třeba 3 epochy)
    # wandb => přidat do modelu
    # train, evaluacni (kontroluje pretrenovani), a pak hold-out test na konec (neukazovat modelu)

https://github.com/huggingface/datasets/issues/610

https://www.analyticsvidhya.com/blog/2021/09/torch-dataset-and-dataloader-early-loading-of-data/

https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

https://www.geeksforgeeks.org/how-to-use-a-dataloader-in-pytorch/

https://deeplizard.com/learn/video/mUueSPmcOBc
https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
https://aigeekprogrammer.com/data-preparation-with-dataset-and-dataloader-in-pytorch/
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [21]:
from transformers import DataCollatorForLanguageModeling

#Inputs are dynamically padded to the maximum length of a batch if they are not all of the same length.
tokenizer.mask_token = "<mask>"
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15 # zakryje 15 % tokenu
)

In [24]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from keras.preprocessing.sequence import pad_sequences
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset

# defining the Dataset class
# https://www.geeksforgeeks.org/self-in-python-class/
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# https://medium.com/swlh/how-to-use-pytorch-dataloaders-to-work-with-enormously-large-text-files-bbd672e955a0
class data_set(Dataset):
    def __init__(self, data, labels, tokenizer):
        self.data = data
        self.labels = labels
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        seq = self.data[index]
        lab = self.labels[index]
        tokenized = tokenizer.encode(seq, max_length=128, padding="max_length", truncation=True)
        # print(tokenized, lab)
        tnsr = torch.tensor(tokenized)
        
        return tnsr # odstranit labely
        
df = pd.read_csv(data_path, sep="\t", names=['type','sequence'])
dataset = data_set(df["sequence"],df["type"], tokenizer)

# def collate_tokenize(list_of_dicts):
#     seqs = [i.get("seq") for i in list_of_dicts]
#     labels = [i.get("labels") for i in list_of_dicts]
#     # tokenized = tokenizer.encode_batch(seqs)
#     tokenized = tokenizer.batch_encode_plus(seqs, padding='longest')
#     ids = [i for i in tokenized.get('input_ids')]
#     return [ids, labels]

# https://github.com/pytorch/pytorch/issues/42654
# https://discuss.huggingface.co/t/huggingface-dataset-each-element-in-list-of-batch-should-be-of-equal-size/10697/3
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_tokenize) #, collate_fn=lambda x: x )
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=data_collator) #, collate_fn=lambda x: x )
# dataloader = DataLoader(dataset, batch_size=64, shuffle=True) #, collate_fn=lambda x: x )
data = next(iter(dataloader))
# data


# display(data)
# data.get("input_ids")
# for (idx, batch) in enumerate(dataloader):
#     ...

In [ ]:
# for epoch in range(2):
#     print("\n==============================\n")
#     print("Epoch = " + str(epoch))
#     for (batch_idx, batch) in enumerate(dataloader):
#         print("\nBatch = " + str(batch_idx))
#         print(batch)
#         break

In [ ]:
# import torch
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
# from keras.preprocessing.sequence import pad_sequences
# from torch.nn.utils.rnn import pad_sequence
# from datasets import load_dataset

# # defining the Dataset class
# # https://www.geeksforgeeks.org/self-in-python-class/
# # https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# # https://medium.com/swlh/how-to-use-pytorch-dataloaders-to-work-with-enormously-large-text-files-bbd672e955a0
# class data_set(Dataset):
#     def __init__(self, data, labels):
#         self.data = data
#         self.labels = labels
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         seq = self.data[index]
#         lab = self.labels[index]
#         return {"seq":seq, "labels":lab}
        
# data_path = "/home/lieberze/DP/Thesis/model_training/data/512_bp_for_encoding/All_equal_shuffled_10k.txt"       
# df = pd.read_csv(data_path, sep="\t", names=['type','sequence'])
# dataset = data_set(df["sequence"],df["type"])

# def collate_tokenize(list_of_dicts):
#     seqs = [i.get("seq") for i in list_of_dicts]
#     labels = [i.get("labels") for i in list_of_dicts]
#     # tokenized = tokenizer.encode_batch(seqs)
#     tokenized = tokenizer.batch_encode_plus(seqs, padding='longest')
#     ids = [i for i in tokenized.get('input_ids')]
#     return [ids, labels]

# # https://github.com/pytorch/pytorch/issues/42654
# # https://discuss.huggingface.co/t/huggingface-dataset-each-element-in-list-of-batch-should-be-of-equal-size/10697/3
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_tokenize) #, collate_fn=lambda x: x )
# data = next(iter(dataloader))
# # data, labels = next(iter(dataloader))
# # data 
# # for (idx, batch) in enumerate(dataloader):
# #     ...

In [ ]:
# for epoch in range(2):
#     print("\n==============================\n")
#     print("Epoch = " + str(epoch))
#     for (batch_idx, batch) in enumerate(dataloader):
#         print("\nBatch = " + str(batch_idx))
#         X, Y = batch        
#         print(X)
#         print(Y)
#         break

odebrat 10% testovací set jeste, holdout

OD KAMILY:
- https://github.com/utkuozbulak/pytorch-custom-dataset-examples
- https://github.com/huggingface/transformers/blob/master/src/transformers/data/datasets/language_modeling.py

In [25]:
df["type"] = pd.Categorical(df["type"])
df['type'] = df.type.cat.codes
# df.type

In [26]:
from sklearn.model_selection import train_test_split

df_train, df_eval = train_test_split(df, test_size=0.25, random_state=42, stratify=df["type"])
df_train, df_holdout = train_test_split(df_train, test_size=0.1, random_state=42, stratify=df_train["type"])

df_train = df_train.reset_index()
df_eval = df_eval.reset_index()
df_holdout = df_holdout.reset_index()

data_set_train = data_set(df_train["sequence"],df_train["type"], tokenizer)
data_set_eval = data_set(df_eval["sequence"],df_eval["type"], tokenizer)
# nesahat :)
holdout_test = data_set(df_holdout["sequence"],df_holdout["type"], tokenizer)

In [27]:
df_train.shape, df_eval.shape

((4505525, 3), (1668714, 3))

In [ ]:
# # PROBLÉM BYL TADY - je potřeba resetnout index
# df_train, df_eval = train_test_split(df, test_size=0.25, random_state=42, stratify=df["type"])
# df_train = df_train.reset_index()
# data_set_train = data_set(df_train["sequence"],df_train["type"], tokenizer)
# dataloader = DataLoader(data_set_train, batch_size=64, shuffle=True, collate_fn=data_collator) #, collate_fn=lambda x: x )
# data = next(iter(dataloader))
# data

In [ ]:
# # https://huggingface.co/docs/transformers/custom_datasets
# from transformers import DataCollatorWithPadding
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [28]:
from transformers import RobertaConfig, RobertaModel
from transformers import RobertaForMaskedLM

# Initializing a RoBERTa configuration
configuration = RobertaConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

model = RobertaForMaskedLM(config=configuration)

model.num_parameters()
# => 47 million parameters - dát do diplomky

47359880

In [ ]:
# 10M radku
# ukazovat loss fci u eval mnoziny
# vyhodit kratke exony? pokud jich je malo
# pospojovat exony (podle genů/chromosomů??), rozsekat podle pravidel
# udělat z toho pak třeba 512, 1024 (lepší pro robertu)
# pak ale zmenšit batche

# pipeline( trénuje se?) -> adapter -> upravit data

# na klasifikaci požívat adapter: == plug-in, model se nemění
# https://adapterhub.ml -> zalozka "training"

# projit si treovaci argumenty:
# learning_rate, weight_decay, adam_epsilon, max_grad_norm
# https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html

In [29]:
# train_size = len(df_train)
# train_size = 2500000

eval_size = len(df_eval)
number_of_epochs = 4
WANTED_eval_data_points_ratio = 0.1
    # proportionally to the whole dataset size. e.g. 0.1 == 10% of all optimization
    # steps is going to have an evaluation datapoint (loss)

train_batch_size = eval_batch_size = 64
total_optimization_steps = len(df_train)/train_batch_size
eval_steps_in_one_run = eval_size/(number_of_epochs*train_batch_size)
eval_steps_to_set=int(round(total_optimization_steps*WANTED_eval_data_points_ratio, 0))

print("total_optimization_steps:",total_optimization_steps,
      "\nhow many 'eval_steps' to set:",eval_steps_to_set,
      "\nhow many steps in each evaluation stop:",eval_steps_in_one_run,
      "\nhence in total:", eval_steps_to_set*eval_steps_in_one_run, "steps for the whole evaluation" )

total_optimization_steps: 70398.828125 
how many 'eval_steps' to set: 7040 
how many steps in each evaluation stop: 6518.4140625 
hence in total: 45889635.0 steps for the whole evaluation


In [30]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta_out_4_epochs_new_tokenizer_",
    overwrite_output_dir=True,
    num_train_epochs=number_of_epochs, # 2-3
    per_device_train_batch_size=train_batch_size, # vetsi batch (128-256 lepsi), ale uvidime
    per_device_eval_batch_size=eval_batch_size,
    save_steps=10_000,
    save_total_limit=2,
        # prediction_loss_only # rozhodne ne!
    seed=1,
    report_to="wandb",
    run_name="bioroberta", #cokoli
    eval_steps = eval_steps_to_set, 
        # evaluation is performed in every n-th Total optimization step = trenink_set/velikost_batche
        # a potom pocet kroku kazdeho stepu evaluace: velikost_eval/(epochy*velikost_batche)
    evaluation_strategy="steps", # evaluates after every "eval_steps" (another option: "epoch")
        # https://happytransformer.com/learning-parameters/
    learning_rate=3e-4, # How much the model’s weights are adjusted per step
    weight_decay=0.0005, # defaults to 0, A type of regularization. It prevents weights from getting too large. Thus, preventing overfitting.
    adam_epsilon=1e-8, # defaults to 1e-8 – Epsilon for the Adam optimizer.
    max_grad_norm=1, # defaults to 1.0 – Maximum gradient norm (for gradient clipping). Used to prevent exploding gradients. 
                    # Prevents the derivatives of the loss function from exceed the absolute value of “max_grad_norm”
    logging_steps=100, 
    warmup_ratio = 0.05,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=data_set_train,
    eval_dataset=data_set_eval, 
    # compute_metrics
)

In [ ]:
%%time
%%wandb

trainer.train()

trainer.save_model("./roberta-trained-new-tokenizer_")

In [ ]:
# result = trainer.train() 
# training_loss = result.training_loss

In [31]:
!nvidia-smi # ukaze kolik pameti to zabralo (na kolika gpu to pocita)

Mon Mar 14 16:56:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:21:00.0 Off |                  N/A |
|100%   26C    P2    63W / 250W |   1566MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:22:00.0 Off |                  N/A |
|100%   

In [32]:
model = torch.load("./roberta-trained-new-tokenizer/pytorch_model.bin")
# model

In [33]:
wandb.finish() 

In [34]:
#https://towardsdatascience.com/transformers-retraining-roberta-base-using-the-roberta-mlm-procedure-7422160d5764
# %reload_ext tensorboard

# %tensorboard --logdir ./runs

In [1]:
# maskování podle tokenů(!), jistota aspon 20% by byla najs

from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./roberta-trained-new-tokenizer_params_4",
    tokenizer=tokenizer
)

intron = "GTGCCACCTGAAGCCTGGTTTGTGTTGGTTATTTCTACTCCTGAACACGTTTTATTTATGGGCATCGAGGCTCACTGTCCATTTGCTCTGTTATCTCTCTGAGTCTTCAGTTCTGGTGCTGACCGACCAGGAATCCTGGCCTTGAGAGCCAGCCTATGACTGCTCTGAGGTACACCTCCTCTGGGGGCTTAGAGCACAGCCCCTGGAGGATGTTGGAAGGGGAAATCCCAGGGGAGATGGGGATGGTGTGACCAGACCCAGTGAGGGCGGGGGGAAATGTCTGGGAAATAACTACCACCAGGCCAGGAGTGagtgagttttttgttttaaatccaaaaaaaaaaaaaaaaaaaaaaaaaagagtccctttccttatttttgtgttttttttcccccttcacttGAGCCACATGTTGCTATGGGAACGAGATGGCACTGTGGCACTGGGACCCACGGTCACTCACTGGCCTGACAGAAGCCCGTAAATAaactctgcctccctccaggctTGG"
tokens = tokenizer.tokenize(intron)
# ['G', 'TGCC', 'ACCTG', 'AAGCC', 'TGG', 'TTTG', 'TGTTGG', 'TTATTTC', 'TACTCC', 'TGAAC', 'ACG', 'TTTT', 'ATTTATG', 'GGC', 'ATCG', 'AGGC', 'TCACTG', 'TCC', 'ATTTGC', 'TCTG', 'TTATC', 'TCTCTG', 'AGTCTTC', 'AGTTC', 'TGG', 'TGCTG', 'ACCG', 'ACCAGG', 'AATCC', 'TGGCC', 'TTG', 'AGAGCC', 'AGCC', 'TATG', 'ACTGC', 'TCTG', 'AGGTAC', 'ACCTCC', 'TCTGGG', 'GGC', 'TTAGAGC', 'ACAG', 'CCCC', 'TGGAGG', 'ATG', 'TTGG', 'AAGGGG', 'AAATCCC', 'AGGGG', 'AGATG', 'GGG', 'ATGG', 'TGTGACC', 'AGACCC', 'AGTG', 'AGGGC', 'GGGGGG', 'AAATG', 'TCTGGG', 'AAATAAC', 'TACC', 'ACC', 'AGGCC', 'AGG', 'AGTG', 'agtgag', 'ttttttg', 'ttttaa', 'atcc', 'aaaaaaaaaaaaaaaa', 'aaaaaaaa', 'aagag', 'tccc', 'tttcc', 'ttatttt', 'tgtg', 'tttttttt', 'cc', 'ccc', 'ttc', 'actt', 'G', 'AGCC', 'ACATG', 'TTGC', 'TATGGG', 'AACG', 'AGATGGC', 'ACTG', 'TGGC', 'ACTGGG', 'ACCC', 'ACGG', 'TCACTC', 'ACTGGCC', 'TGACAG', 'AAG', 'CCCG', 'TAA', 'ATA', 'aac', 'tctgcc', 'tccctcc', 'aggc', 't', 'TGG']
intron_masked = "GTGCC<mask>AAGCCTGGTTTGTGTTGGTTATTTCTACTCCTGAACACGTTTTATTTATGGGCATCGAGGCTCACTGTCCATTTGCTCTGTTATCTCTCTGAGTCTTCAGTTCTGGTGCTGACCGACCAGGAATCCTGGCCTTGAGAGCCAGCCTATGACTGCTCTGAGGTACACCTCCTCTGGGGGCTTAGAGCACAGCCCCTGGAGGATGTTGGAAGGGGAAATCCCAGGGGAGATGGGGATGGTGTGACCAGACCCAGTGAGGGCGGGGGGAAATGTCTGGGAAATAACTACCACCAGGCCAGGAGTGagtgagttttttgttttaaatccaaaaaaaaaaaaaaaaaaaaaaaaaagagtccctttccttatttttgtgttttttttcccccttcacttGAGCCACATGTTGCTATGGGAACGAGATGGCACTGTGGCACTGGGACCCACGGTCACTCACTGGCCTGACAGAAGCCCGTAAATAaactctgcctccctccaggctTGG"
fill_mask(intron_masked)

exon = "CATCACCAGATAATCCAGCAGTACATGGTGACCCTGGAGAACCTGCTCTTCACGGCAGAGTTAGATCCTCACATCCTGGCTGTGTTCCAGCAGTTCTGTGCTCTGCAGGCCTAACAGTCACGTTTTCCCCTTTTTATTATCTTGAAATAACTTGTCTGAATTTTGATGGTTTGAATGTTTGCTTTCCTGTAGTAGCCTTTCACTTCCtaaaggaaatgggggaggagtGTGAGGAACATGAGCTGACCGACCTTTAGTTGCCCTGAAGGGCAAGTGTTACTCGATGACAGTGATGTTATGGTGACTGTAGAGTTGTACATATGGGATAGATTTGATACTTTTTCTCCTTAGTGCACAGAACAAGAGGTTTGAGGTGTTCGAAATGCAAGGTCAGGCTTGCAAAATAGTAGATACAACCGACAGGCTATCCCCGGGGAGCGTAGCTTTTCTCTGCTCCTGGGAAACATGGCTGGAATAAACTGCAGTTGTGTTTTTTAAA"
tokens = tokenizer.tokenize(exon)
# ['C', 'ATCACC', 'AGATAA', 'TCCAGC', 'AGTAC', 'ATGG', 'TGACCC', 'TGG', 'AGAACC', 'TGC', 'TCTTC', 'ACGGC', 'AGAG', 'TTAG', 'ATCC', 'TCAC', 'ATCC', 'TGGC', 'TGTG', 'TTCCAGC', 'AGTTCTG', 'TGC', 'TCTGC', 'AGGCC', 'TAAC', 'AGTC', 'ACG', 'TTTTCCCC', 'TTTT', 'TATT', 'ATCTTG', 'AAAT', 'AACTTG', 'TCTG', 'AATTTTG', 'ATGG', 'TTTG', 'AATG', 'TTTGC', 'TTTCC', 'TGTAG', 'TAGCC', 'TTTC', 'ACTTCC', 'taaagg', 'aaatg', 'gggg', 'aggag', 't', 'GTG', 'AGGAAC', 'ATG', 'AGCTG', 'ACCG', 'ACC', 'TTTAG', 'TTGCCC', 'TG', 'AAGGGC', 'AAG', 'TGTT', 'ACTCG', 'ATG', 'ACAG', 'TGATG', 'TTATGG', 'TGACTG', 'TAG', 'AGTTG', 'TAC', 'ATATGGG', 'ATAG', 'ATTTG', 'ATAC', 'TTTT', 'TCTCC', 'TT', 'AGTGC', 'ACAGAAC', 'AAGAGG', 'TTTGAGG', 'TGTTC', 'G', 'AAATGC', 'AAGG', 'TCAGGC', 'TTGC', 'AAAA', 'TAGTAG', 'ATACAA', 'CCG', 'ACAGGC', 'TATCCCC', 'GGGG', 'AGCG', 'TAGC', 'TTTTC', 'TCTGC', 'TCCTGGG', 'AAAC', 'ATGGC', 'TGG', 'AATAA', 'ACTGC', 'AGTTG', 'TGTTTT', 'TTAA', 'A']
exon_masked = "CATCACC<mask>TCCAGCAGTACATGGTGACCCTGGAGAACCTGCTCTTCACGGCAGAGTTAGATCCTCACATCCTGGCTGTGTTCCAGCAGTTCTGTGCTCTGCAGGCCTAACAGTCACGTTTTCCCCTTTTTATTATCTTGAAATAACTTGTCTGAATTTTGATGGTTTGAATGTTTGCTTTCCTGTAGTAGCCTTTCACTTCCtaaaggaaatgggggaggagtGTGAGGAACATGAGCTGACCGACCTTTAGTTGCCCTGAAGGGCAAGTGTTACTCGATGACAGTGATGTTATGGTGACTGTAGAGTTGTACATATGGGATAGATTTGATACTTTTTCTCCTTAGTGCACAGAACAAGAGGTTTGAGGTGTTCGAAATGCAAGGTCAGGCTTGCAAAATAGTAGATACAACCGACAGGCTATCCCCGGGGAGCGTAGCTTTTCTCTGCTCCTGGGAAACATGGCTGGAATAAACTGCAGTTGTGTTTTTTAAA"
fill_mask(exon_masked)

NameError: name 'tokenizer' is not defined

In [36]:
# # ! pip install bertviz
# ! pip uninstall transformers
# ! pip uninstall adapter-transformers

In [65]:
# https://colab.research.google.com/drive/1hXIQ77A4TYS4y3UthWF-Ci7V7vVUoxmQ?usp=sharing#scrollTo=TG-dQt3NOlub
from bertviz import head_view, model_view
import os
from transformers import AutoModelWithHeads

Transformers = "/home/lieberze/DP/Thesis/05_model_training/"
TransformerName = os.path.abspath(os.path.join(Transformers, "roberta-trained-new-tokenizer_params_4")) 

model_version = TransformerName
model = AutoModelWithHeads.from_pretrained(model_version, output_attentions=True)
tokenizer = tokenizer
sentence_a = "ATGTTTGCTTTCCTGTAGTAGCCTTTCACTTCCtaaaggaaatgggggaggagtGTGAGGAACATGAGCTGACCGACCTTTAG"
# sentence_b = "TAGTGCACAGAACAAGAGGTTTGAGGTGTTCGAAATGCAAGGTCAGGCTTGCAAAATAGTAGATACAACCGACAGGCTATCCCCGGGGAGCGTAGCTTTTCTCTGCTCCTGGGAAACATGGC"
inputs = tokenizer.encode_plus(sentence_a, return_tensors='pt') # sentence_b
input_ids = inputs['input_ids']
# token_type_ids = inputs['token_type_ids']
attention = model(input_ids)[-1]
# attention = model(input_ids, token_type_ids=token_type_ids)[-1]

# sentence_b_start = token_type_ids[0].tolist().index(1)
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list) 
print(input_id_list, tokens)

head_view(attention, tokens)
# model_view(attention, tokens, sentence_b_start)

loading configuration file /home/lieberze/DP/Thesis/05_model_training/roberta-trained-new-tokenizer_params_4/config.json
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {},
    "fusion_config_map": {},
    "fusions": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_attentions": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 5000
}

loading weights file /home/lieberze/DP/Thesis/05_model_training/roberta-train

[0, 3175, 275, 539, 2005, 339, 298, 439, 2575, 644, 545, 1349, 4686, 490, 348, 638, 528, 572, 302, 2] ['<s>', 'ATGTTTG', 'CTT', 'TCCTG', 'TAGTAG', 'CCTT', 'TCA', 'CTTCC', 'taaagg', 'aaatg', 'gggg', 'aggag', 'tGTG', 'AGGAA', 'CATG', 'AGCTG', 'ACCG', 'ACCTT', 'TAG', '</s>']


<IPython.core.display.Javascript object>

In [ ]:
# sweeps (grid na trenovani parametru transformeru? - learning_rate...)
# vic dat a vic epoch
# prelozit graf trenovaci + evaluacni
# pridat epochu, data, upravit hyperparametry
# proverit balancovana data

In [ ]:
# # https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=75SNIg6ekAtV
# # https://www.youtube.com/watch?v=9zrmUIlScdY
# sweep_config = {
#     'method': 'random'
#     }

# metric = {
#     'name': 'loss',
#     'goal': 'minimize'   
#     }

# sweep_config['metric'] = metric


In [ ]:
# param_grid = {
#     "per_gpu_batch_size": [16, 32],
#     "learning_rate": [2e-5, 3e-5, 5e-5],
#     "num_epochs": [2, 3, 4],
#     "weight_decay": (0, 0.3),
# }

In [ ]:
# # https://github.com/ray-project/ray/blob/master/python/ray/tune/examples/pbt_transformers/pbt_transformers.py
# tune_config = {
#         "per_device_train_batch_size": 32,
#         "per_device_eval_batch_size": 32,
#         "num_train_epochs": tune.choice([2, 3, 4, 5]),
#         "max_steps": 1 if smoke_test else -1,  # Used for smoke test.
#     }
    
# trainer.hyperparameter_search(
#         hp_space=lambda _: tune_config,
#         backend="ray",
#         n_trials=num_samples,
#         resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
#         scheduler=scheduler,
#         keep_checkpoints_num=1,
#         checkpoint_score_attr="training_iteration",
#         stop={"training_iteration": 1} if smoke_test else None,
#         progress_reporter=reporter,
#         local_dir="~/ray_results/",
#         name="tune_transformer_pbt",
#         log_to_file=True,
#     )

In [ ]:
# # historie trénování modelu
# trainer.state.log_history

### adding adapter for classification - potřeba udělat v dalším notebooku a env (kvůli adapter-transformers)

In [ ]:
# # from transformers import RobertaTokenizer, RobertaConfig#,# RobertaModelWithHeads
# from transformers import AutoModelWithHeads

# # tokenizer = RobertaTokenizer.from_pretrained(
# #     "roberta-base"
# # )
# config = RobertaConfig.from_pretrained(
#     "roberta-trained",
#     num_labels=3,
#     id2label={ 0: "exon", 1: "intron", 2:"intergenic"},
# )
# model = AutoModelWithHeads.from_pretrained(
#     "roberta-trained",
#     config=config,
# )

In [ ]:
# model.add_adapter("x")
# adapter_name = model.load_adapter("rotten")
# model.active_adapters = adapter_name

In [ ]:
import transformers
# from transformers import AutoModelWithHeads
# model = AutoModelWithHeads.from_pretrained("roberta-trained")

In [ ]:
# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained("roberta-trained")
# model.load_adapter("sentiment/sst-2@ukp")
# model.set_active_adapters("sst-2")

In [ ]:
# model

In [ ]:
# # Add a new adapter
# model.add_adapter("rotten_tomatoes")
# # # Add a matching classification head
# # model.add_classification_head(
# #     "rotten_tomatoes",
# #     num_labels=2,
# #     id2label={ 0: "👎", 1: "👍"}
# #   )
# # # Activate the adapter
# # model.train_adapter("rotten_tomatoes")

In [ ]:
# model = AutoModelWithHeads.from_pretrained('./roberta-trained"')
# model.add_adapter("sst-2")
# model.train_adapter("sst-2")